# Rigid Body Simulator

Credit to Professor Bo Zhu, on whose COSC 89 Course Notes the following code is based on

## Geometry in space

In [205]:
import taichi as ti
import numpy as np

ti.init()

# variables and fields to keep track of
num_particles = 1
dt = 1e-3
substeps = 10
mass = 1

# Inertial tensor
I = 10
# position - center of mass
x = ti.Vector.field(n=2, dtype=ti.f32, shape=num_particles)
# x = ti.Vector([0.5, 0.5])
# vertices - relative to center of mass -- will have to change if more than one solid
vertices_0 = ti.Vector.field(n=2, dtype=ti.f32, shape=num_particles*3)
rotated_vertices = ti.Vector.field(n=2, dtype=ti.f32, shape=num_particles*3)
# force
force = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
# torque
torque = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
# velocity
v = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
# orientation
theta = ti.field(dtype=ti.f32, shape=num_particles)
# angular velocity
w = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)

@ti.kernel
def init_pos():
    x[0] = ti.Vector([0.5, 0.5])
    # equilateral triangle
    vertices_0[0] = ti.Vector([0, 0.1])
    vertices_0[1] = ti.Vector([-0.0866, -0.05])
    vertices_0[2] = ti.Vector([0.0866, -0.05])
    rotated_vertices[0] = ti.Vector([0, 0.1])
    rotated_vertices[1] = ti.Vector([-0.0866, -0.05])
    rotated_vertices[2] = ti.Vector([0.0866, -0.05])
    # initialize velocities
    v[0] = ti.Vector([0, 0, 0])
    # initialize force
    force[0] = ti.Vector([0,0,0])
    
@ti.func
def clear_force():
    for i in range(num_particles):
        force[i] = ti.Vector([0.0, 0.0, 0.0])
    
    
@ti.func
def calculate_force():
    clear_force()
    # gravity
    for i in range(num_particles):
        force[i] = force[i] - ti.Vector([0, -2, 0])

@ti.func
def rotate():
    pass

@ti.func
def calculate_torque():
    pass

@ti.kernel
def substep():
    calculate_force()
    calculate_torque()
    # update velocity
    v[0] = v[0] + dt * force[0] / mass
    # update angular velocity
    w[0] = w[0] + dt * torque[0] / I
    # update position
    x[0] = x[0] + dt * v[0]
    # update orientation
    theta[0] = theta[0] * dt * w[0]
    # update vertices
    rotate()

def main():
    gui = ti.GUI('Rigid Body', (600, 600))
    # initialize vertices
    init_pos()
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            pass
            
        # Update for a time step
        for step in range(substeps):
            substep()

        gui.triangle(a=x[0]+rotated_vertices[0], b=x[0]+rotated_vertices[1], c=x[0]+rotated_vertices[2], color=0xED553B)
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64


TaichiCompilationError: 
On line 76 of file "/var/folders/y7/rxbvhmr9307f345mkrl28_xw0000gn/T/ipykernel_5370/1594515034.py", in substep:
    x[0] = x[0] + dt * v[0]
           ^^^^^^^^^^^^^^^^
Traceback (most recent call last):
  File "/Users/andresibarra/opt/anaconda3/lib/python3.9/site-packages/taichi/lang/ast/ast_transformer_utils.py", line 24, in __call__
    return method(ctx, node)
  File "/Users/andresibarra/opt/anaconda3/lib/python3.9/site-packages/taichi/lang/ast/ast_transformer.py", line 683, in build_BinOp
    node.ptr = op(node.left.ptr, node.right.ptr)
  File "/Users/andresibarra/opt/anaconda3/lib/python3.9/site-packages/taichi/lang/ast/ast_transformer.py", line 669, in <lambda>
    ast.Add: lambda l, r: l + r,
  File "/Users/andresibarra/opt/anaconda3/lib/python3.9/site-packages/taichi/lang/common_ops.py", line 35, in __add__
    return ops.add(self, other)
  File "/Users/andresibarra/opt/anaconda3/lib/python3.9/site-packages/taichi/lang/ops.py", line 55, in wrapped
    r

Code from stackoverflow on finding the inertial tensor using triangles

https://stackoverflow.com/questions/41592034/computing-tensor-of-inertia-in-2d

[Taichi] Starting on arch=x64


## Being able to move particles with mouse

In [ ]:
import taichi as ti

ti.init()

max_particles = 100
num_particles = ti.field(dtype=ti.i32, shape=())
dt = 1e-3
substeps = 10
screen_size = 400

ks = 2e2
kd = 0.5e1

x = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
r = ti.field(dtype=ti.f32, shape=max_particles)
m = ti.field(dtype=ti.f32, shape=max_particles)


@ti.kernel
def substep():
    # find force
    for i in range(num_particles[None]):
        f[i] = [0,-2] # reset force, add in gravity
        # boundary check
        phi = x[i][1] - r[i]/screen_size
        if phi < 0: # lower boundary
            f[i] += ks * (phi) * ti.Vector([0,-1])
            f[i] += kd * (ti.Vector([0,0]) - v[i]).dot(ti.Vector([0,-1])) * ti.Vector([0, -1])
        phi = x[i][0] - r[i]/screen_size
        if phi < 0: # left boundary
            f[i] += ks * (phi) * ti.Vector([-1,0])
            f[i] += kd * (ti.Vector([0,0]) - v[i]).dot(ti.Vector([-1,0])) * ti.Vector([-1,0])
        phi = x[i][0] + r[i]/screen_size
        if phi > 1:
            f[i] += ks * (phi) * ti.Vector([-1,0])
            f[i] -= kd * (ti.Vector([0,0]) - v[i]).dot(ti.Vector([-1,0])) * ti.Vector([-1,0])
            ###
            # How do I make it so the resting state isn't slightly submerged
            ###

        for j in range(i+1, num_particles[None]):
            # check for collision
            phi = (x[i] - x[j]).norm() - r[i]/screen_size - r[j]/screen_size
            normal = (x[j] - x[i]).normalized()
            if phi < 0:
                force_spring = ks * (phi) * normal
                force_damp = kd * (v[j] - v[i]).dot(normal) * normal
                f[i] += force_spring
                f[j] -= force_spring
                f[i] += force_damp
                f[j] -= force_damp
                
    for i in range(num_particles[None]):
        # update v
        v[i] = v[i] + dt * f[i] / m[i]
        # update x
        x[i] = x[i] + dt * v[i]
        
@ti.kernel
def new_particle(pos_x: ti.f32, pos_y: ti.f32):
    particle_id = num_particles[None]
    if particle_id < max_particles:
        x[particle_id] = [pos_x, pos_y]
        v[particle_id] = [0, 0]
        m[particle_id] = 1
        r[particle_id] = 10
        num_particles[None] += 1
        

class MouseDataGen:
    def __init__(self):
        self.prev_mouse = None

    def __call__(self, gui):
        # [0:2]: normalized delta direction
        # [2:4]: current mouse xy
        # [4:7]: color
        mouse_data = np.zeros(8, dtype=np.float32)
        if gui.is_pressed(ti.GUI.LMB):
            mxy = np.array(gui.get_cursor_pos(), dtype=np.float32) * res
            if self.prev_mouse is None:
                self.prev_mouse = mxy
            else:
                mdir = mxy - self.prev_mouse
                mdir = mdir / (np.linalg.norm(mdir) + 1e-5)
                mouse_data[0], mouse_data[1] = mdir[0], mdir[1]
                mouse_data[2], mouse_data[3] = mxy[0], mxy[1]
                self.prev_mouse = mxy
        else:
            self.prev_mouse = None
        return mouse_data
        
        
def main():
    gui = ti.GUI('Particle Sand', (screen_size, screen_size))
    
    new_particle(0.3, 0.5)
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == ti.GUI.UP:
                ks *= 1.1
            elif e.key == ti.GUI.DOWN:
                ks /= 1.1
            elif e.key == ti.GUI.LMB:
                new_particle(e.pos[0], e.pos[1])
            
        # Move stuff
        for step in range(substeps):
            substep()
             
        # Draw particles
        X = x.to_numpy()
        for i in range(num_particles[None]):
            gui.circle(pos=X[i], radius = r[i])

        # show the current values
        gui.text(
            content='Click anywhere on screen to add a new mass',
            pos=(0,0.99),
            color=0xffffff)
        
        gui.show()
    
if __name__ == '__main__':
    main()

In [ ]:
@ti.kernel
def advect(vf: ti.template(), qf: ti.template(), new_qf: ti.template()):
    for i, j in vf:
        p = ti.Vector([i, j]) + 0.5
        p = backtrace(vf, p, dt)
        new_qf[i, j] = bilerp(qf, p) * dye_decay

@ti.kernel
def apply_impulse(vf: ti.template(), dyef: ti.template(),
                  imp_data: ti.types.ndarray()):
    g_dir = -ti.Vector([0, 9.8]) * 300
    for i, j in vf:
        omx, omy = imp_data[2], imp_data[3]
        mdir = ti.Vector([imp_data[0], imp_data[1]])
        dx, dy = (i + 0.5 - omx), (j + 0.5 - omy)
        d2 = dx * dx + dy * dy
        # dv = F * dt
        factor = ti.exp(-d2 / force_radius)

        dc = dyef[i, j]
        a = dc.norm()

        momentum = (mdir * f_strength * factor + g_dir * a / (1 + a)) * dt

        v = vf[i, j]
        vf[i, j] = v + momentum
        # add dye
        if mdir.norm() > 0.5:
            dc += ti.exp(-d2 * (4 / (res / 15)**2)) * ti.Vector(
                [imp_data[4], imp_data[5], imp_data[6]])

        dyef[i, j] = dc
        

def step(mouse_data):
    advect(velocities_pair.cur, velocities_pair.cur, velocities_pair.nxt)
    advect(velocities_pair.cur, dyes_pair.cur, dyes_pair.nxt)
    velocities_pair.swap()
    dyes_pair.swap()

    apply_impulse(velocities_pair.cur, dyes_pair.cur, mouse_data)


class MouseDataGen:
    def __init__(self):
        self.prev_mouse = None

    def __call__(self, gui):
        # [0:2]: normalized delta direction
        # [2:4]: current mouse xy
        # [4:7]: color
        mouse_data = np.zeros(8, dtype=np.float32)
        if gui.is_pressed(ti.GUI.LMB):
            mxy = np.array(gui.get_cursor_pos(), dtype=np.float32) * res
            if self.prev_mouse is None:
                self.prev_mouse = mxy
            else:
                mdir = mxy - self.prev_mouse
                mdir = mdir / (np.linalg.norm(mdir) + 1e-5)
                mouse_data[0], mouse_data[1] = mdir[0], mdir[1]
                mouse_data[2], mouse_data[3] = mxy[0], mxy[1]
                self.prev_mouse = mxy
        else:
            self.prev_mouse = None
        return mouse_data


def main():
    paused = False

    gui = ti.GUI('Stable Fluid', (res, res))
    md_gen = MouseDataGen()

    while gui.running:
        if not paused:
            mouse_data = md_gen(gui)
            step(mouse_data)
